# Solid-Liquid Coexistence Method for Melting Point Calculation with GPUMD

This notebook provides a step-by-step guide for using GPUMD to calculate the melting point of alloys via the solid-liquid coexistence (SLC) method. The workflow is compared to LAMMPS-based structure generation, but focuses on the GPUMD simulation and analysis.

## 1. Objective

Determine the melting point of Pd-Cu-Ni-P alloys using the solid-liquid coexistence method in GPUMD. The method involves equilibrating a pre-constructed SLC structure at various temperatures and identifying the melting point from a sharp jump in the potential energy vs. temperature curve.

## 2. Required Dependencies

- GPUMD (compiled executable)
- NEP potential file (e.g., `nep.txt`)
- Python 3.x (for analysis and plotting)
- numpy, matplotlib (for data analysis)
- Initial structure files (`model.*.xyz`) generated by LAMMPS

**Note:** NEP model training is not covered here; you need a pre-trained NEP file.

## 3. Workflow Overview

1. **Structure Generation (LAMMPS):** Use LAMMPS to generate SLC structures (`model.*.xyz`).
2. **Initial GPUMD Relaxation:** Relax the SLC structure at low and high temperatures to obtain solid-liquid coexistence configurations.
3. **Melting Point Calculation:** Equilibrate the system at a series of temperatures using GPUMD and analyze the potential energy curve.

## 4. Model Preparation

- `model.*.xyz`: SLC structure files for each composition.
- `nep.txt`: NEP potential file for Pd-Cu-Ni-P alloys.

These files should be present in the working directory.

In [ ]:
# Check for required files
import os
required_files = ['nep.txt'] + [f'model.{e}.xyz' for e in ['Pd', 'Cu', 'Ni', 'P']]
for f in required_files:
    print(f, 'exists:' if os.path.exists(f) else 'MISSING')

## 5. GPUMD Simulation Setup

The GPUMD input file (`run.in`) for initial relaxation typically includes:

```plaintext
potential       ../../0bak/nep.txt
velocity        T1
time_step       1
ensemble        npt_scr T1 T1 100 0 150 1000
fix             1
dump_thermo     1000
dump_exyz       100000 0 0
dump_restart    100000
run             1000000

ensemble        npt_scr T2 T2 100 0 150 1000
fix             0
dump_thermo     1000
dump_exyz       100000 0 0
dump_restart    100000
run             1000000
```

Where T1 and T2 are the low and high temperatures for relaxation.

## 6. Running GPUMD

To run the simulation, use the following command in your shell (PowerShell on Windows):

```powershell
gpumd.exe > log.txt 2>&1
```

Or use the provided shell scripts (`1init.sh`, `2MP.sh`) on Linux/Mac.

After completion, output files like `thermo.out` and `restart.xyz` will be generated.

## 7. Melting Point Calculation: Potential Energy Analysis

For each temperature, calculate the average potential energy from the last 1/10 of simulation steps in `thermo.out`. Plot the potential energy vs. temperature curve to identify the melting point.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Example: Load PE vs T data for Pd
data = np.loadtxt('PEvsT.Pd.txt')  # Columns: T, PE/atom
T = data[:,0]
PE = data[:,1]
plt.figure(figsize=(8,5))
plt.plot(T, PE, marker='o')
plt.xlabel('Temperature (K)')
plt.ylabel('Potential Energy per Atom (eV)')
plt.title('Potential Energy vs. Temperature (Pd)')
plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Iterative Refinement

If no sharp jump is observed, redefine the temperature range and rerun the simulations to narrow down the melting point. Repeat until the melting point is accurately determined.

## 9. Summary

This notebook guides you through the solid-liquid coexistence method for melting point calculation using GPUMD. The workflow includes structure preparation, simulation setup, execution, and analysis. For further customization, adjust the input files and simulation parameters as needed.